In [18]:
import os
import pandas as pd
from datetime import datetime

In [19]:
current_date = datetime.now().strftime("%d_%m_%Y")
print(current_date)

folder_name = f"dashboard_files_{current_date}"
folder_path = os.path.join(r"C:\Users\O304312\Documents\Dashboard Files", folder_name)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at: {folder_path}")
else:
    print(f"Folder already exists: {folder_path}")

22_09_2025
Folder already exists: C:\Users\O304312\Documents\Dashboard Files\dashboard_files_22_09_2025


In [20]:
today = datetime.today()
first_of_this_month = today.replace(day=1)
first_of_last_month = (first_of_this_month - pd.DateOffset(months=1)).replace(day=1)

month = first_of_last_month.strftime("%B").lower()
print(month)
cap_month = month.capitalize()
print(cap_month)

august
August


In [21]:
snapshot_ds = r"J:\ONELINK\Snapshot Datasources"

In [22]:
visit_data = pd.read_csv(
    r"C:\Users\O304312\Downloads\Data Table Visit Data.csv",
    low_memory=False,
)

print(visit_data.columns)

Index(['Unnamed: 0', 'Site Name', 'Site Study Code', 'Subject ID',
       'Patient MRN', 'Patient Name', 'Patient Birth Date',
       'Patient Phone Number', 'Patient Email Address', 'Address',
       'Participant Status', 'Site Protocol Version',
       'Participant Protocol Arm', 'Visit Period', 'Visit Name',
       'Visit Location', 'Visit Status', 'Enrolling Investigator',
       'Enrolling Coordinator', 'Active Investigator', 'Visit Scheduled Date',
       'Visit Target Date', 'Unplanned Visit Y N', 'Visit Completed Date',
       'Latest Visit Note', 'Site Study PI', 'Lead Research Coordinator'],
      dtype='object')


In [23]:
visit_data = visit_data[
    [
        "Site Name",
        "Site Study Code",
        "Subject ID",
        "Participant Status",
        "Participant Protocol Arm",
        "Visit Period",
        "Visit Name",
        "Visit Status",
        "Visit Completed Date",
    ]
]
print(visit_data.dtypes)

visit_data = visit_data.rename(
    columns={
        "Site Name": "Site",
        "Site Study Code": "Site Study ID",
        "Participant Protocol Arm": "Arm",
    }
)

# Convert Visit Completed Date to datetime
visit_data["Visit Completed Date"] = pd.to_datetime(
    visit_data["Visit Completed Date"], errors="coerce"
)

visit_data = visit_data[visit_data["Visit Completed Date"] < first_of_last_month]

visit_data.to_csv(f"{folder_path}/Visit Data_data.csv", index=False)

Site Name                   object
Site Study Code             object
Subject ID                  object
Participant Status          object
Participant Protocol Arm    object
Visit Period                object
Visit Name                  object
Visit Status                object
Visit Completed Date        object
dtype: object


In [24]:
study_accruals = pd.read_csv(
    r"C:\Users\O304312\Downloads\Data Table Study Accrual.csv",
    low_memory=False,
)

print(study_accruals.columns)

Index(['Unnamed: 0', 'Site Name', 'With Location', 'Sponsor Site Number',
       'Nct ID', 'Network Study Code', 'Network Study Status',
       'Site Study Code', 'Site Study Status', 'Study Title', 'Study Phase',
       'Study Type', 'Study Purpose', 'Funding Source', 'Site IRB Name',
       'Site IRB Status', 'Site IRB Expiration Date', 'IRB Submission Number',
       'Study Account Code', 'Principal Investigators', 'Lead Coordinators',
       'Regulatory Coordinators', 'Site Study Start Date',
       'Site Study End Date', 'Site Enrollment Start Date',
       'Site Enrollment End Date', 'Study Therapeutic Areas',
       'Study Therapeutic Area Details', 'Study Sponsors',
       'Site Enrollment Target', 'CRO Name', 'Total Patients Prescreened Sum',
       'Failed Prescreening Sum', 'Total Participants Sum', 'In Screening Sum',
       'Failed Screening Sum', 'In Treatment Sum', 'Completed Sum',
       'Follow Up Sum', 'Off Study Sum', 'Long Term Follow Up Sum'],
      dtype='object')

In [25]:
study_accruals = study_accruals[
    [
        "Site Name",
        "Sponsor Site Number",
        "Network Study Code",
        "Network Study Status",
        "Site Study Code",
        "Site Study Status",
        "Study Title",
        "Study Phase",
        "Study Type",
        "Funding Source",
        "Site IRB Name",
        "Site IRB Status",
        "Site IRB Expiration Date",
        "IRB Submission Number",
        "Principal Investigators",
        "Lead Coordinators",
        "Site Study Start Date",
        "Site Study End Date",
        "Site Enrollment Start Date",
        "Site Enrollment End Date",
        "Study Therapeutic Areas",
        "Study Therapeutic Area Details",
        "Study Sponsors",
        "Site Enrollment Target",
        "CRO Name",
        "Total Patients Prescreened Sum",
        "Failed Prescreening Sum",
        "Total Participants Sum",
        "In Screening Sum",
        "Failed Screening Sum",
        "In Treatment Sum",
        "Completed Sum",
        "Follow Up Sum",
        "Off Study Sum",
        "Long Term Follow Up Sum",
    ]
]

study_accruals = study_accruals.rename(
    columns={
        "Site Name": "Site",
        "Site Study Code": "Site Study ID",
        "Site Enrollment Target": "Site Enrollment Targets",
    }
)

print(study_accruals.dtypes)

study_accruals.to_csv(f"{folder_path}/Study Accrual Data.csv", index=False)

Site                               object
Sponsor Site Number               float64
Network Study Code                 object
Network Study Status               object
Site Study ID                      object
Site Study Status                  object
Study Title                        object
Study Phase                        object
Study Type                         object
Funding Source                     object
Site IRB Name                      object
Site IRB Status                    object
Site IRB Expiration Date           object
IRB Submission Number             float64
Principal Investigators            object
Lead Coordinators                  object
Site Study Start Date              object
Site Study End Date                object
Site Enrollment Start Date         object
Site Enrollment End Date           object
Study Therapeutic Areas            object
Study Therapeutic Area Details     object
Study Sponsors                    float64
Site Enrollment Targets           

In [26]:
partis_info = pd.read_csv(
    r"C:\Users\O304312\Downloads\Data Table Participant Information.csv",
    low_memory=False,
)

print(partis_info.columns)

Index(['Unnamed: 0', 'Site Name', 'Participant Location', 'Patient Name',
       'Patient MRN', 'Subject ID', 'Site Study Code', 'Study Type',
       'Study Purpose', 'Participant Status', 'Participant Status Date',
       'Patient Birth Date', 'Patient Age Num', 'Sex At Birth',
       'Patient Gender Identity', 'Patient Ethnicity', 'Patient Race',
       'Participant Initial Consent Date', 'Participant Latest Consent Date',
       'Participant Consent Status', 'Participant Eligibility Date',
       'Participant Eligibility Result', 'Screening Date',
       'Total Screen Failures', 'Latest Screen Failure Date',
       'Latest Screen Failure Notes', 'Participant Enrollment Date',
       'Participant Enrolled (Yes / No)', 'Treatment Date', 'Follow Up Date',
       'Off Study Date', 'Completed Date', 'Long Term Follow Up (Yes / No)',
       'Address', 'Patient Phone Number', 'Patient Email Address',
       'Study Therapeutic Areas', 'Patient Therapeutic Areas',
       'Study Therapeutic A

In [27]:
partis_info = partis_info[
    [
        "Subject ID",
        "Site Study Code",
        "Participant Status",
        "Participant Status Date",
        "Participant Latest Consent Date",
        "Participant Consent Status",
    ]
]

partis_info["Participant Latest Consent Date"] = partis_info[
    "Participant Latest Consent Date"
].replace(["", "0", 0], pd.NA)
if "Latest Screen Failure Date" in partis_info.columns:
    partis_info["Participant Latest Consent Date"] = partis_info[
        "Participant Latest Consent Date"
    ].fillna(partis_info["Latest Screen Failure Date"])


partis_info = partis_info.rename(
    columns={
        "Site Study Code": "Site Study ID",
        "Participant Status Date": "Current Status Date",
        "Participant Latest Consent Date": "Current Consent Date",
        "Participant Consent Status": "Consent Result",
    }
)

partis_info["Current Consent Date"] = pd.to_datetime(
    partis_info["Current Consent Date"], errors="coerce"
)

partis_info = partis_info[partis_info["Current Consent Date"] < first_of_last_month]

partis_info.to_csv(
    f"{folder_path}/Participant Information Data Sheet.csv",
    index=False,
)

In [28]:
transactions = pd.read_csv(
    r"C:\Users\O304312\Downloads\Transaction.csv", low_memory=False
)

transactions = transactions.drop(columns=["Unnamed: 0"])
print(transactions.columns)

Index(['Transaction Created Date', 'Accountable Completed Date', 'Aging Date',
       'Accountable ID', 'Payment Date', 'Transaction Line Item Type',
       'Transaction Line Item', 'Protocol Version', 'Holdback Percentage',
       'Holdback Amount', 'Invoiceable (Yes / No)', 'Payment ID', 'Provider',
       'Vendor', 'Vendor Code', 'Account Code', 'Sub Account Code',
       'Department Code', 'Study Account Code', 'Service Line Code',
       'Grant Code', 'Site Account Code', 'Network Revenue Expense Code',
       'Activity Code', 'Transaction Amount', 'Payment Amount', 'Patient Name',
       'Patient MRN', 'Routine Care (Yes / No)', 'Transaction Type',
       'Ledger Entry Type', 'AR/AP Status', 'Site Name', 'Site Study Code',
       'Subject ID', 'Sponsor Study ID', 'CRO Name', 'Visit Name',
       'Participant Protocol Arm', 'Visit Location', 'Event Name',
       'Sponsor Name', 'Principal Investigators', 'Lead Coordinators'],
      dtype='object')


In [29]:
transactions["Accountable Completed Date"] = pd.to_datetime(
    transactions["Accountable Completed Date"], errors="coerce"
)

transactions["Accountable Completed Date"] = transactions[
    "Accountable Completed Date"
].replace(["", "0", 0], pd.NA)

transactions = transactions[
    (transactions["Accountable Completed Date"] < first_of_last_month)
    | (transactions["Accountable Completed Date"].isnull())
]

mask = transactions["Accountable Completed Date"].isna()
transactions.loc[mask, "Accountable Completed Date"] = pd.to_datetime(
    transactions.loc[mask, "Transaction Created Date"], errors="coerce"
)

num_blank_rows = transactions["Accountable Completed Date"].isna().sum()
print(f"Number of blank rows in 'Accountable Completed Date': {num_blank_rows}")

transactions = transactions.drop(columns=["Transaction Created Date"])

transactions["Account Code"] = transactions["Account Code"].replace(["", "0", 0], pd.NA)

transactions["Account Code"] = transactions["Account Code"].fillna(47205)

transactions.loc[
    transactions["Transaction Line Item"] == "StudyActivity", "Account Code"
] = 47206


# Next Steps: merge with DOR data in chen file to get overhead rate and then multiply all adhoc changes by oh rate to update cost

Number of blank rows in 'Accountable Completed Date': 836


In [30]:
import glob
import re

excel_files = glob.glob(
    r"J:\ADMIN-eFILES\CHEN_W154867_VXC\zzz_CTP Projects List\DOR Finance CTP List\CTP Project List Reconciled *.xlsx"
)


def extract_date(filename):
    match = re.search(r"(\d{6})\.xlsx$", filename)
    return match.group(1) if match else ""


excel_files_sorted = sorted(excel_files, key=extract_date, reverse=True)

if excel_files_sorted:
    latest_excel = excel_files_sorted[0]
    dor_ctp_projects = pd.read_excel(latest_excel)
    print(f"Loaded file: {latest_excel}")
else:
    print("No matching Excel files found.")

Loaded file: J:\ADMIN-eFILES\CHEN_W154867_VXC\zzz_CTP Projects List\DOR Finance CTP List\CTP Project List Reconciled 250916.xlsx


In [31]:
print(transactions.columns)
print(dor_ctp_projects.columns)
dor_ctp_projects = dor_ctp_projects[["ProjectID", "IDC Rate"]]

dor_ctp_projects = dor_ctp_projects.drop_duplicates()

Index(['Accountable Completed Date', 'Aging Date', 'Accountable ID',
       'Payment Date', 'Transaction Line Item Type', 'Transaction Line Item',
       'Protocol Version', 'Holdback Percentage', 'Holdback Amount',
       'Invoiceable (Yes / No)', 'Payment ID', 'Provider', 'Vendor',
       'Vendor Code', 'Account Code', 'Sub Account Code', 'Department Code',
       'Study Account Code', 'Service Line Code', 'Grant Code',
       'Site Account Code', 'Network Revenue Expense Code', 'Activity Code',
       'Transaction Amount', 'Payment Amount', 'Patient Name', 'Patient MRN',
       'Routine Care (Yes / No)', 'Transaction Type', 'Ledger Entry Type',
       'AR/AP Status', 'Site Name', 'Site Study Code', 'Subject ID',
       'Sponsor Study ID', 'CRO Name', 'Visit Name',
       'Participant Protocol Arm', 'Visit Location', 'Event Name',
       'Sponsor Name', 'Principal Investigators', 'Lead Coordinators'],
      dtype='object')
Index(['Contract ID', 'ProjectID', 'Functional \nArea', 'Proj

In [32]:
transactions = pd.merge(
    transactions,
    dor_ctp_projects,
    left_on="Service Line Code",
    right_on="ProjectID",
    how="left",
)

transactions["Transaction Amount"] = pd.to_numeric(
    transactions["Transaction Amount"], errors="coerce"
)
transactions["IDC Rate"] = pd.to_numeric(transactions["IDC Rate"], errors="coerce")

mask = (transactions["Transaction Line Item"] == "AdHoc") & (
    transactions["Account Code"] == 47205
)
transactions.loc[mask, "Transaction Amount"] = transactions.loc[
    mask, "Transaction Amount"
] * (1 + transactions.loc[mask, "IDC Rate"].fillna(0))

In [33]:
project_id_list = pd.read_csv(
    "J:\\ONELINK//Snapshot Datasources//SignalPath ProjectID Lookup.csv",
    low_memory=False,
    encoding="cp1252",
)
print(project_id_list.columns)

transactions = pd.merge(
    transactions,
    project_id_list,
    left_on="Service Line Code",
    right_on="Project ID",
    how="left",
)

print(transactions.columns)

list_csv = pd.read_csv(
    r"J:\ONELINK\Snapshot Datasources\04 2025 April\_Transaction Data april 2025.csv"
)

columns_list = list_csv.columns.tolist()

print(columns_list)
transactions["Transaction Created Date"] = transactions["Accountable Completed Date"]
transactions = transactions.rename(
    columns={
        "Transaction Amount": "Amount",
        "Site Study ID": "Site Protocol Version Desc",
    }
)

transactions = transactions.loc[:, columns_list]

Index(['Site Study ID', 'Project ID'], dtype='object')
Index(['Accountable Completed Date', 'Aging Date', 'Accountable ID',
       'Payment Date', 'Transaction Line Item Type', 'Transaction Line Item',
       'Protocol Version', 'Holdback Percentage', 'Holdback Amount',
       'Invoiceable (Yes / No)', 'Payment ID', 'Provider', 'Vendor',
       'Vendor Code', 'Account Code', 'Sub Account Code', 'Department Code',
       'Study Account Code', 'Service Line Code', 'Grant Code',
       'Site Account Code', 'Network Revenue Expense Code', 'Activity Code',
       'Transaction Amount', 'Payment Amount', 'Patient Name', 'Patient MRN',
       'Routine Care (Yes / No)', 'Transaction Type', 'Ledger Entry Type',
       'AR/AP Status', 'Site Name', 'Site Study Code', 'Subject ID',
       'Sponsor Study ID', 'CRO Name', 'Visit Name',
       'Participant Protocol Arm', 'Visit Location', 'Event Name',
       'Sponsor Name', 'Principal Investigators', 'Lead Coordinators',
       'ProjectID', 'IDC Rate

C:\Users\O304312\AppData\Local\Temp\ipykernel_5980\2156164570.py:18: DtypeWarning: Columns (1,2,4,7,9,10,12,15,21,22,25,26,27,30,32,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  list_csv = pd.read_csv(


['Site Name', 'Transaction Created Date', 'Accountable Completed Date', 'Aging Date', 'Accountable ID', 'Payment Date', 'Transaction Line Item Type', 'Transaction Line Item', 'Site Study Code', 'Site Protocol Version Desc', 'AR/AP Status', 'Holdback Percentage', 'Holdback Amount', 'Invoiceable (Yes / No)', 'Payment ID', 'Provider', 'Vendor', 'Vendor Code', 'Account Code', 'Sub Account Code', 'Department Code', 'Study Account Code', 'Service Line Code', 'Grant Code', 'Site Account Code', 'Network Revenue Expense Code', 'Activity Code', 'Amount', 'Payment Amount', 'Patient Name', 'Patient MRN', 'Routine Care (Yes / No)', 'Transaction Type', 'Ledger Entry Type', 'Subject ID', 'Sponsor Study ID', 'CRO Name', 'Visit Name', 'Participant Protocol Arm', 'Visit Location', 'Event Name', 'Sponsor Name', 'Principal Investigators', 'Lead Coordinators']


In [ ]:
transactions.to_csv(
    f"{folder_path}/_Transaction Data.csv",
    index=False,
)